# 1) Packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re, string

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.metrics.pairwise import cosine_similarity

bold = "\033[1m"

# 2) Load Data

In [ ]:
episodes = pd.read_csv("../input/south-park-scripts-dataset/SouthPark_Episodes.csv")

episodes.head()

In [ ]:
lines = pd.read_csv("../input/south-park-scripts-dataset/SouthPark_Lines.csv")

lines = lines[lines["Line"].notnull()]
lines

# 3) Joing Scripts and Characters

In [ ]:
episodes_list = episodes.Title.tolist()

script = {}

for episode in episodes_list:
    
    temp = lines[lines["Title"] == episode]
    episode_script = " ".join(temp.Line)
    script[episode] = episode_script
    
full_script_df = pd.DataFrame(script.items(), columns = ["Title", "Script"])
full_script_df

In [ ]:
characters = {}

for episode in episodes_list:
    temp = lines[lines["Title"] == episode]
    episode_characters = " ".join(temp.Character) 
    characters[episode] = episode_characters
    
full_characters_df = pd.DataFrame(characters.items(), columns = ["Title", "Characters"])
full_characters_df

# 4) Vectorizing Characters, Scripts and Descriptions

For similarity, we don't have to use all characters. Setting a minimum line limit helps us for speeding our calculations and getting better results.

In [ ]:
cast = lines.Character.value_counts()[lines.Character.value_counts() > 50].index.tolist()

cv = CountVectorizer(lowercase = False)

characters = cv.fit_transform(full_characters_df["Characters"])
characters_cv = pd.DataFrame(characters.todense(), columns = cv.get_feature_names())

characters_cv.set_index(full_characters_df.Title, inplace = True)

characters_cv = characters_cv[[x for x in characters_cv.columns if x in cast]]

characters_cv

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have",
"wanna": "want to",
"gonna": "going to",
"gotta": "have got to"
}

In [ ]:
all_stopwords = nlp.Defaults.stop_words

def tokenizer(text):
    
    text = text.replace("in'", "ing")
    text = text.replace("m'kay", "mkay")
    tokens = text.split()
    tokens = [re.sub(token, contractions[token], token) if token in contractions.keys() else token for token in tokens]
    tokens = [token.strip(string.punctuation) for token in tokens]    
    tokens = [token.lower() for token in tokens]    
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if len(token) > 1]
    tokens = [token for token in tokens if token not in all_stopwords]

#     n = lambda pos: pos[:2].startswith("N")
#     tokens = [word for (word, pos) in nltk.pos_tag(tokens) if n(pos)] 
    
    lemmatizer = nltk.wordnet.WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token, "v")  if token != "butters" else "butters" for token in tokens]
    lemmas = [lemmatizer.lemmatize(token)  if token != "butters" else "butters" for token in lemmas]
    
    return lemmas

In [ ]:
tfidf_script = TfidfVectorizer(min_df = 3, max_df = 0.8, tokenizer = tokenizer, ngram_range = (1, 3), max_features = 5000,
#                         binary = True, use_idf = False, norm = None
                       )

tfidf_script_matrix = tfidf_script.fit_transform(full_script_df["Script"])
tfidf_script_df = pd.DataFrame(tfidf_script_matrix.todense(), columns = tfidf_script.get_feature_names())

tfidf_script_df.set_index(full_script_df.Title, inplace = True)

tfidf_script_df

In [ ]:
tfidf_description = TfidfVectorizer(min_df = 2, max_df = 0.8, tokenizer = tokenizer, ngram_range = (1, 3), max_features = 5000,
#                         binary = True, use_idf = False, norm = None
                       )

tfidf_description_matrix = tfidf_description.fit_transform(episodes["Description"])
tfidf_description_df = pd.DataFrame(tfidf_description_matrix.todense(), columns = tfidf_description.get_feature_names())

tfidf_description_df.set_index(episodes.Title, inplace = True)

tfidf_description_df

If we don't scale, characters_cv frame will be more important. Other features, scripts and descriptions will not affect so much.

In [ ]:
scaler = MinMaxScaler()

for col in characters_cv.columns:
    characters_cv[col] = scaler.fit_transform(characters_cv[col].values.reshape(-1, 1))

In [ ]:
train = pd.concat([characters_cv, tfidf_script_df, tfidf_description_df], axis = 1)
train

# 5) Recommend Episodes, or Find Similar Episodes

In [ ]:
cosine_sim = cosine_similarity(train) #Recommendations for using characters, lines, and description of episodes
cosine_sim_chars = cosine_similarity(characters_cv) #Recommendations for using characters number of lines of episodes
cosine_sim_lines = cosine_similarity(tfidf_script_df) #Recommendations for using script of that episode
cosine_sim_description = cosine_similarity(tfidf_description_df) #Recommendations for using description of that episode

indices = pd.Series(range(0, len(train.index)), index = train.index).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim = cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    recommendations = pd.DataFrame({"Episodes": train.iloc[movie_indices].index.tolist(),
                                    "Similarity": [sim[1] for sim in sim_scores]})
    return recommendations

**Recommendations for just using characters that have lines:**

In [ ]:
get_recommendations("Mr. Hankey, the Christmas Poo", cosine_sim_chars).head(5)

**Chef's Chocolate Salty Balls** could be a good recommendation, but others... They would better.

**Recommendations for using script of that episode:**

In [ ]:
get_recommendations("Mr. Hankey, the Christmas Poo", cosine_sim_lines).head(5)

Nice, all recommendations are good for Mr. Hankey lovers.

**Recommendations for using description of that episode:**

In [ ]:
get_recommendations("Mr. Hankey, the Christmas Poo", cosine_sim_description).head(5)

It looks like we are fail. We get irrelevant recommendations.

**Recommendations for using characters, lines, and description of episode:**

In [ ]:
get_recommendations("Mr. Hankey, the Christmas Poo").head(5)

Concatenating all features gives good result. However recommendations with using lines could be better.

In [ ]:
print(bold + "With using all features: \n")
display(get_recommendations("Terrance and Phillip: Behind the Blow").head(5))

print(bold + "With using just scripts: \n")
display(get_recommendations("Terrance and Phillip: Behind the Blow", cosine_sim_lines).head(5))

In [ ]:
print(bold + "With using all features: \n")
display(get_recommendations("Tweek x Craig").head(5))

print(bold + "With using just scripts: \n")
display(get_recommendations("Tweek x Craig", cosine_sim_lines).head(5))

In [ ]:
print(bold + "With using all features: \n")
display(get_recommendations("Skank Hunt").head(5))

print(bold + "With using just scripts: \n")
display(get_recommendations("Skank Hunt", cosine_sim_lines).head(5))

In [ ]:
get_recommendations("Butters' Very Own Episode")

Generally Butters is main character of that episodes.